In [89]:
# pip install -U classiq

In [90]:
# import classiq
# classiq.authenticate()

In [ ]:
import numpy as np
from classiq import *
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from collections import defaultdict
from typing import List

from classiq.execution import (
    ClassiqBackendPreferences,
    ClassiqSimulatorBackendNames,
    ExecutionSession,
    ExecutionPreferences,
)

np.random.seed(0)  # Please don't change

# Define the 2D Hamiltonian
HAMILTONIAN = [
    PauliTerm([Pauli.I, Pauli.I], -1.0523),
    PauliTerm([Pauli.I, Pauli.Z], 0.3979),
    PauliTerm([Pauli.Z, Pauli.I], -0.3979),
    PauliTerm([Pauli.Z, Pauli.Z], -0.0112),
    PauliTerm([Pauli.X, Pauli.X], 0.1809),
]

hamiltonian_matrix = hamiltonian_to_matrix(HAMILTONIAN)
print(hamiltonian_matrix)

[[-1.0635+0.j  0.    +0.j  0.    +0.j  0.1809+0.j]
 [ 0.    +0.j -1.8369+0.j  0.1809+0.j  0.    +0.j]
 [ 0.    +0.j  0.1809+0.j -0.2453+0.j  0.    +0.j]
 [ 0.1809+0.j  0.    +0.j  0.    +0.j -1.0635+0.j]]


In [92]:
def analytical_solution(hamiltonian):
    """
    Compute the analytical solution of the Hamiltonian.

    Returns:
        - eig_val: a list of eigenvalues
        - eig_vec: a list of eigenvectors
    """
    Hamiltonian_matrix = hamiltonian_to_matrix(hamiltonian)

    eig_val, eig_vec = np.linalg.eig(Hamiltonian_matrix)

    return eig_val, eig_vec.transpose()

def normalize(amps):
    return amps / np.linalg.norm(amps)

def get_uniform_superposition(eig_vecs):
    """
    Compute the uniform superposition of the eigenvectors.
    Args:
        - eig_vecs: a list of eigenvectors

    Returns:
        - vec: a list representing the uniform superposition of the eigenvectors
    """
    N = len(eig_vecs)
    
    # Ensure there is at least one eigenvector
    if N == 0:
        return []

    #transform each eig_vec into np.array()
    eig_vecs = [np.array(eig_vec) for eig_vec in eig_vecs]

    # make sum and multiply by 1/sqrt(N)
    superposition = np.sum(eig_vecs, axis=0) / np.sqrt(N)

    return normalize(superposition)

In [93]:
eig_val, eig_vecs = analytical_solution(HAMILTONIAN)

for i in range(len(eig_vecs)):
    print(f"Eigenvalue {i}: {eig_val[i]}")
    print(f"Eigenvector {i}: {eig_vecs[i]}")

uniform_superposition = get_uniform_superposition(eig_vecs)

print("Uniform superposition", uniform_superposition)

for i in range(len(eig_vecs)):
    inner_product = np.abs(np.dot(uniform_superposition, eig_vecs[i]))**2
    print(f"inner superposition with eigenvec {i}: {inner_product}")

Eigenvalue 0: (-0.8826+0j)
Eigenvector 0: [ 0.70710678+0.j -0.        -0.j -0.        -0.j  0.70710678-0.j]
Eigenvalue 1: (-1.2444000000000002+0j)
Eigenvector 1: [ 0.70710678+0.j  0.        +0.j  0.        +0.j -0.70710678+0.j]
Eigenvalue 2: (-0.22499801495156269+0j)
Eigenvector 2: [-0.        -0.j  0.11152752-0.j  0.99376135+0.j -0.        -0.j]
Eigenvalue 3: (-1.8572019850484374+0j)
Eigenvector 3: [-0.        -0.j  0.99376135+0.j -0.11152752-0.j -0.        -0.j]
Uniform superposition [ 7.07106781e-01+0.j  5.52644432e-01+0.j  4.41116914e-01+0.j
 -5.55111512e-17+0.j]
inner superposition with eigenvec 0: 0.2500000000000002
inner superposition with eigenvec 1: 0.2500000000000001
inner superposition with eigenvec 2: 0.25
inner superposition with eigenvec 3: 0.24999999999999994


In [94]:
np.random.seed(0)  # Please don't change

def get_t_n(T_RMS, N, tol=0.5):
    """
    Generate a list of N positive random numbers from a normal distribution with a given RMS value.
    The function repeats the generation until the RMS of the generated numbers is within a tolerance of tol.

    Args:
        T_RMS: Desired RMS value.
        N: Number of random numbers to generate.
        tol: Tolerance for the RMS difference (default: 1).
        
    Returns:
        t_n: A list of N positive random numbers.
    """
    while True:
        t_n = np.abs(np.random.normal(loc=0, scale=T_RMS, size=N)).tolist()
        calculated_rms = np.sqrt(np.mean(np.square(t_n)))
        if abs(calculated_rms - T_RMS) < tol:
            print(f"Calculated RMS (T_RMS={T_RMS}): {calculated_rms}")
            return t_n

def aggregate_counts(sample_result, N):
    """
    Aggregate the counts from the sample results.
    """
    for i, item in enumerate(sample_result):
        counts = item.counts
        aggregated = {}
        for key, value in counts.items():
            # Only the last N characters correspond to the ancilla register configuration
            agg_key = key[-N:]
            aggregated[agg_key] = aggregated.get(agg_key, 0) + value
    return aggregated

In [95]:
@qfunc
def trotter(qbv: QArray[QBit], t: CInt):
    """
    Apply the Suzuki-Trotter decomposition to time evolution of the object Hamiltonian.
    """
    operator = [PauliTerm(term.pauli, term.coefficient * t) for term in HAMILTONIAN]
    suzuki_trotter(
        pauli_operator=operator,
        evolution_coefficient=1,
        order=1,
        repetitions=30,
        qbv=qbv
    )

@qfunc
def apply_control(ctrl: QBit, target: QArray, t: CInt):
    """
    Apply the controlled Suzuki-Trotter decomposition.
    """
    control(ctrl=ctrl, stmt_block=(lambda: trotter(target, t=t)))

In [106]:
N = 6
T_RMS = 20
E=-1.2444000000000002
initial_vec = uniform_superposition

@qfunc
def main(a: Output[QArray], x: Output[QArray], E: CReal, t_n: CArray[CReal, 6]):
    """
    Main quantum function implementing the Rodeo algorithm.
    """
    prepare_amplitudes(amplitudes=initial_vec, bound=0.01, out=x)
    allocate(N, a)
    # Prepare ancilla in the |+> state (using |0> state as default)
    hadamard_transform(a)
    # Apply controlled trotter evolution
    repeat(N, lambda i: apply_control(a[i], x, t=t_n[i]))
    # Apply the phase rotation
    repeat(N, lambda i: phase(a[i], E*t_n[i]))
    hadamard_transform(a)

qmod = create_model(main)
qprog = synthesize(qmod)
t_n = get_t_n(T_RMS, N)
data = [{"E": E, "t_n": t_n}]

execution_preferences = ExecutionPreferences(
    backend_preferences = ClassiqBackendPreferences(
        backend_name=ClassiqSimulatorBackendNames.SIMULATOR_STATEVECTOR
    )
)
with ExecutionSession(
    qprog, execution_preferences=execution_preferences
) as session:
    results = session.batch_sample(data)

print(results)

Calculated RMS (T_RMS=20): 19.820908790148465
[ExecutionDetails(vendor_format_result={}, counts={'10110100': 1, '11011111': 1, '01110010': 2, '01000001': 1, '10010011': 2, '11110011': 2, '11110110': 2, '01000101': 4, '00010100': 1, '11110100': 8, '00011011': 1, '11011101': 2, '11011011': 1, '01010101': 1, '10000000': 3, '10100111': 17, '11110000': 35, '11111010': 6, '11111000': 26, '00000000': 261, '00011001': 7, '10100101': 24, '10000101': 189, '00010001': 21, '10110111': 5, '11010000': 20, '10000111': 66, '00011101': 3, '11000000': 248, '11011001': 14, '10000011': 4, '10100100': 4, '00110011': 2, '01110101': 73, '00111001': 41, '00010101': 6, '01110000': 12, '11110010': 1, '11011000': 8, '01110100': 385, '10010101': 64, '11011100': 2, '10110001': 3, '00011100': 4, '10000100': 15, '00111100': 5, '01010100': 1, '10001101': 1, '01000111': 1, '11111001': 24, '00110000': 33, '00110001': 40, '00111000': 30, '00110101': 8, '11111101': 11, '01110110': 38, '00110100': 9, '11111100': 8, '11010

In [120]:
from classiq import *
from classiq.execution import (
    ClassiqBackendPreferences,
    ExecutionSession,
    ExecutionPreferences,
    ClassiqSimulatorBackendNames,
)

N=1
T_RMS = 20
E=-1.2444000000000002
t_n = get_t_n(T_RMS, N)
data = [{"E": E, "t_n": t_n}]

@qfunc
def main(a: Output[QArray], x: Output[QArray], E: CReal, t_n: CArray[CReal, 1]):
    prepare_state(probabilities=[1, 0, 0, 0], bound=0.01, out=x)
    allocate(N, a)
    hadamard_transform(a)
    repeat(N, lambda i: apply_control(a[i], x, t=t_n[i]))
    repeat(N, lambda i: phase(a[i], E*t_n[i]))
    hadamard_transform(a)

quantum_program = synthesize(main)

execution_preferences = ExecutionPreferences(
    backend_preferences = ClassiqBackendPreferences(
        backend_name=ClassiqSimulatorBackendNames.SIMULATOR_STATEVECTOR
    )
)
with ExecutionSession(
    quantum_program, execution_preferences=execution_preferences
) as session:
    results = session.batch_sample(data)

print(results)

Calculated RMS (T_RMS=20): 19.71021475368301
[ExecutionDetails(vendor_format_result={}, counts={'111': 92, '001': 70, '000': 1792, '110': 94}, counts_lsb_right=True, probabilities={}, parsed_states={'111': {'a': [1], 'x': [1, 1]}, '001': {'a': [1], 'x': [0, 0]}, '000': {'a': [0], 'x': [0, 0]}, '110': {'a': [0], 'x': [1, 1]}}, histogram=None, output_qubits_map={'a': (0,), 'x': (1, 2)}, state_vector={'000': (0.45181284518850284+0.817881485681526j), '001': (-0.1537381789194751+0.1366610714255696j), '010': (1.0100179356041244e-15+2.5353251851757205e-15j), '011': (6.150264125707835e-16+3.70422767096001e-15j), '100': (7.140412785324301e-16+8.04824393134031e-15j), '101': (-2.940120581615324e-16+8.133494402468304e-15j), '110': (0.15373817891947386-0.13666107142556083j), '111': (-0.15373817891947422+0.1366610714255612j)}, parsed_state_vector_states={'000': {'a': [0], 'x': [0, 0]}, '001': {'a': [1], 'x': [0, 0]}, '010': {'a': [0], 'x': [1, 0]}, '011': {'a': [1], 'x': [1, 0]}, '100': {'a': [0], '